In [1]:
pip install fuzzywuzzy


In [2]:
import pickle
from fuzzywuzzy import process, fuzz

class ProductNameMapper:
    def __init__(self, mapping_file="mapping_dict.pkl"):
        self.mapping_file = mapping_file
        try:
            # Load existing mapping dictionary if available
            with open(self.mapping_file, 'rb') as file:
                self.mapping_dict = pickle.load(file)
        except FileNotFoundError:
            # Initialize with an empty dictionary if file not found
            self.mapping_dict = {}

    def save_mapping(self):
        """Save the mapping dictionary to file."""
        with open(self.mapping_file, 'wb') as file:
            pickle.dump(self.mapping_dict, file)

    def manual_map(self, raw_name, standardized_name):
        """Manually map a raw product name to a standardized name."""
        self.mapping_dict[raw_name.lower()] = standardized_name
        self.save_mapping()
        print(f"Added mapping: '{raw_name}' -> '{standardized_name}'")

    def auto_map(self, raw_name):
        """Automatically map a product name if possible."""
        # Normalize input
        raw_name_lower = raw_name.lower()
        if raw_name_lower in self.mapping_dict:
            return self.mapping_dict[raw_name_lower]

        # Fuzzy match suggestions
        suggestions = process.extract(raw_name, self.mapping_dict.keys(), scorer=fuzz.token_sort_ratio, limit=3)
        print(f"Suggestions for '{raw_name}': {suggestions}")
        return None

    def handle_new_name(self, raw_name):
        """Handle an unmapped product name."""
        mapped_name = self.auto_map(raw_name)
        if mapped_name:
            return mapped_name

        print(f"No direct match found for '{raw_name}'. Please provide a standardized name.")
        standardized_name = input("Enter standardized name: ")
        self.manual_map(raw_name, standardized_name)
        return standardized_name

    def display_mappings(self):
        """Display the current mapping dictionary."""
        for raw, std in self.mapping_dict.items():
            print(f"{raw} -> {std}")

# Test the system
if __name__ == "__main__":
    mapper = ProductNameMapper()

    # Existing test cases
    test_products = [
        "A4 Paper 500sh", "A4 Copy Paper 500 sheets", "500 Sheets A4",
        "Sticky Notes 3x3", "Post-it Notes 3x3 Yellow", "3x3 Yellow Sticky Notes"
    ]

    # Standardized names for testing
    standardized_names = {
        "A4 Paper 500sh": "A4 Paper 500 Sheets",
        "A4 Copy Paper 500 sheets": "A4 Paper 500 Sheets",
        "500 Sheets A4": "A4 Paper 500 Sheets",
        "Sticky Notes 3x3": "Yellow Sticky Notes",
        "Post-it Notes 3x3 Yellow": "Yellow Sticky Notes",
        "3x3 Yellow Sticky Notes": "Yellow Sticky Notes"
    }

    # Adding initial mappings
    for raw_name, std_name in standardized_names.items():
        mapper.manual_map(raw_name, std_name)

    # Handle new test cases
    new_test = ["A4 500 sheets", "Yellow 3x3 Post-it"]
    for product in new_test:
        result = mapper.handle_new_name(product)
        print(f"'{product}' mapped to '{result}'")

    # Display all mappings
    print("\nCurrent Mappings:")
    mapper.display_mappings()


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Added mapping: 'A4 Paper 500sh' -> 'A4 Paper 500 Sheets'
Added mapping: 'A4 Copy Paper 500 sheets' -> 'A4 Paper 500 Sheets'
Added mapping: '500 Sheets A4' -> 'A4 Paper 500 Sheets'
Added mapping: 'Sticky Notes 3x3' -> 'Yellow Sticky Notes'
Added mapping: 'Post-it Notes 3x3 Yellow' -> 'Yellow Sticky Notes'
Added mapping: '3x3 Yellow Sticky Notes' -> 'Yellow Sticky Notes'
Suggestions for 'A4 500 sheets': [('500 sheets a4', 100), ('a4 copy paper 500 sheets', 70), ('a4 paper 500sh', 59)]
No direct match found for 'A4 500 sheets'. Please provide a standardized name.
Enter standardized name: A4 Paper 500 Sheets
Added mapping: 'A4 500 sheets' -> 'A4 Paper 500 Sheets'
'A4 500 sheets' mapped to 'A4 Paper 500 Sheets'
Suggestions for 'Yellow 3x3 Post-it': [('post-it notes 3x3 yellow', 86), ('3x3 yellow sticky notes', 73), ('sticky notes 3x3', 53)]
No direct match found for 'Yellow 3x3 Post-it'. Please provide a standardized name.
Enter standardized name: Yellow Sticky Notes
Added mapping: 'Yellow 